In [43]:
from tempfile import mkstemp
import os
import base64

In [44]:
import dataiku
import pandas as pd

In [48]:
import dataikuapi

In [45]:
emotion_videos = dataiku.Folder("JV4cw0cu")
emotion_videos_info = emotion_videos.get_info()

In [73]:
df = pd.DataFrame()
df['video_path'] = emotion_videos.list_paths_in_partition()

def get_emotion(fname):
    code = fname.split('-')[2]

    if code == '01':
        return 'neutral'
    elif code == '02':
        return 'calm'
    elif code == '03':
        return 'happy'
    elif code == '04':
        return 'sad'
    elif code == '05':
        return 'angry'
    elif code == '06':
        return 'fearful'
    elif code == '07':
        return 'disgust'
    elif code == '08':
        return 'surprised'

def get_intensity(fname):
    code = fname.split('-')[3]

    return 'normal' if code == '01' else 'strong'

df['emotion'] = df.video_path.apply(get_emotion)
df['intensity'] = df.video_path.apply(get_intensity)
df['statement'] = df.video_path.apply(lambda s: s.split('-')[4])
df['repetition'] = df.video_path.apply(lambda s: s.split('-')[5])
df['actor'] = df.video_path.apply(lambda s: s.split('-')[6].split('.')[0])

df = df.sample(5)

df

,video_path,emotion,intensity,statement,repetition,actor
991,/02-01-05-01-02-02-17.mp4,angry,normal,02,02,17
1396,/02-01-03-02-01-01-24.mp4,happy,strong,01,01,24
672,/02-01-03-01-01-01-12.mp4,happy,normal,01,01,12
331,/02-01-05-01-02-02-06.mp4,angry,normal,02,02,06
170,/02-01-07-02-02-01-03.mp4,disgust,strong,02,01,03


In [74]:
records = []

for ind, row in df.iterrows():

    video_path, video_folder = row.video_path, emotion_videos

    vf, vfname = mkstemp(suffix='.mp4')

    vfile = os.fdopen(vf, 'wb')

    p = video_path

    with video_folder.get_download_stream(p) as stream:
        vfile.write(stream.read())
    
    vfile.close()
    
    with open(vfname, 'rb') as f:
        b64f = base64.urlsafe_b64encode(f.read())
        
    records.append( {'features': {
        'file_name': row.video_path[1:],
        'b64_video': b64f.decode('utf-8')
    }})

In [75]:
%%time

client = dataikuapi.APINodeClient("http://DATAIKU_API_IP:PORT", "phdata_score_video")
predictions = client.predict_records('prod_score_video', records)

CPU times: user 57.5 ms, sys: 47.4 ms, total: 105 ms
Wall time: 1min 23s


In [76]:
[(e, p['prediction']) for e, p in zip(df.iloc[:,1], predictions['results'])]

[('angry', u'angry'),
 ('happy', u'happy'),
 ('happy', u'happy'),
 ('angry', u'angry'),
 ('disgust', u'disgust')]

In [77]:
[p['probas'] for p in predictions['results']]

[{u'angry': 0.9999194948025025,
  u'calm': 3.232538625803425e-11,
  u'disgust': 7.214155810056381e-09,
  u'fearful': 2.899261839436638e-08,
  u'happy': 9.665126452269528e-10,
  u'neutral': 5.4477839551866e-07,
  u'sad': 7.990829927534331e-05,
  u'surprised': 1.4914214330719474e-08},
 {u'angry': 1.0809065256695846e-08,
  u'calm': 0.13907040559935968,
  u'disgust': 6.099896972649114e-06,
  u'fearful': 9.632139942210348e-07,
  u'happy': 0.860762124740155,
  u'neutral': 0.00013057463829695208,
  u'sad': 6.672459989296258e-08,
  u'surprised': 2.975437755624895e-05},
 {u'angry': 5.1235589388057426e-09,
  u'calm': 4.7758598699899895e-06,
  u'disgust': 7.446455707128955e-10,
  u'fearful': 3.727733135805268e-09,
  u'happy': 0.9999923616959331,
  u'neutral': 7.572230103672934e-07,
  u'sad': 2.1882264087947464e-07,
  u'surprised': 1.8768026080875852e-06},
 {u'angry': 0.9999885225433065,
  u'calm': 1.341651312817087e-07,
  u'disgust': 3.110469520910396e-10,
  u'fearful': 1.2229252310315148e-09,
  